<a href="https://colab.research.google.com/github/Abdullasaqib/DL-Projects/blob/main/Copy_of_Breezy_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
#@title Run this cell to generate the data
! pip install faker
import numpy as np
import pandas as pd
import uuid
import datetime

from faker.providers.person.en import Provider
from faker import Faker

from faker import Faker
fake = Faker()
Faker.seed(0)

n_employees = 50
first_names = pd.Series(list(set(Provider.first_names))).sample(n_employees)
df_employees = pd.DataFrame(columns=["id", "name", "salary"])
df_employees["name"] = (first_names).sample(frac=1).values
mu = 28_000
minimum = 16_000
df_employees["salary"] = (np.random.pareto(a=6, size=(50,1)) * mu) + minimum
df_employees["salary"] = (df_employees["salary"]/100).astype(int) * 100
df_employees["id"] = pd.Series([str(uuid.uuid4()).split('-')[1] for x in range(n_employees)])

def create_random_project(number_of_employees, duration, year, is_ongoing=False):
    df_projects = pd.DataFrame(index=range(number_of_employees), columns=["id", "project_id", "start_date", "end_date"])

    employees = df_employees.id.sample(number_of_employees)

    start_date = datetime.date(year=year, month=1, day=1)
    end_date = datetime.date(year=year+1, month=1, day=1)
    project_start_date = fake.date_between(start_date=start_date, end_date=end_date)
    project_end_date = project_start_date + datetime.timedelta(days=duration)

    if is_ongoing:
        project_end_date = None

    project_id = str(uuid.uuid4()).split('-')[4]

    df_projects["id"] = employees.values
    df_projects["project_id"] = project_id
    df_projects["start_date"] = project_start_date
    df_projects["end_date"] = project_end_date


    return df_projects

n_projects = 150
projects = []
for proj in range(n_projects):
    number_of_employees = np.random.choice([1,2,3,4], replace=True, p=[0.25, 0.5, 0.15, 0.1], size=1)
    duration = (np.random.pareto(a=5, size=1) * 30*4) + 14
    year = np.random.choice([2017, 2018, 2019, 2020])
    is_ongoing = bool(np.random.binomial(n=1, p=0.4)) if year >= 2019 else False
    new_project = (create_random_project(number_of_employees=number_of_employees[0], duration=duration[0], year=year, is_ongoing=is_ongoing).values)

    projects.append(new_project)

projects_data = np.vstack(projects)
df_projects = (
    pd.DataFrame(projects_data, columns=["id", "project_id", "start_date", "end_date"])
    .sort_values(["project_id", "start_date", "end_date"])
    .reset_index(drop=True)
)

In [32]:
df_employees.head(5)

,id,name,salary
0,9f9f,Gerda,26000
1,666a,Narcissus,22700
2,5aec,Todd,21300
3,a526,Denese,20100
4,b12b,Quintin,17400


In [33]:
df_projects.head(5)

,id,project_id,start_date,end_date
0,883a,021a298ce928,2019-11-13,None
1,62a6,021a298ce928,2019-11-13,None
2,474a,021a298ce928,2019-11-13,None
3,f2f6,049d69650cfd,2017-05-28,2017-06-13
4,6610,049d69650cfd,2017-05-28,2017-06-13


## Find the 3 lowest paid employees who have done at least 10 projects.

## Write your solution here, and copy it back to the form

---



In [37]:
# Merge employee data with salary data
merged_df = pd.merge(df_employees, df_projects, on='id')

# Ensure 'Projects' column is numeric
merged_df['project_id'] = pd.to_numeric(merged_df['project_id'], errors='coerce')

# Filter employees who have done at least 10 projects
filtered_df = merged_df.query('project_id >= 10')

# Find the 3 lowest-paid employees
lowest_paid_employees = filtered_df.nsmallest(3, 'salary')

print(lowest_paid_employees)

       id     name  salary    project_id  start_date    end_date
149  ae07    Candi   19500  4.623832e+11  2017-11-05  2017-11-20
222  0b7c    Alois   23100  4.623832e+11  2017-11-05  2017-11-20
164  cf0c  Katelin   23400  4.623832e+11  2017-11-05  2017-11-20
